In [ ]:
import pandas as pd
import os
import kagglehub
drive_path = "/content/drive/MyDrive/SANDBOX/"

# Download latest version
path = kagglehub.dataset_download("carlosaguayo/usa-public-schools")

print("Path to dataset files:", path)

100%|██████████| 11.2M/11.2M [00:01<00:00, 9.78MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/carlosaguayo/usa-public-schools/versions/2


In [ ]:
df = pd.read_csv(path + "/Public_Schools.csv")
df

,X,Y,OBJECTID,NCESID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,ST_GRADE,END_GRADE,DISTRICTID,FT_TEACHER,SHELTER_ID
0,-81.050895,29.022271,2002,120192008041,SAMSULA ACADEMY,248 N SAMSULA DR,NEW SMYRNA,FL,32168,8762,...,IMAGERY,2014-05-20T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,222,KG,05,1201920,13,NOT AVAILABLE
1,-92.507288,31.180659,2003,220129002344,CAROLINE DORMON JUNIOR HIGH SCHOOL,8906 HWY 165 SOUTH,WOODWORTH,LA,71485,NOT AVAILABLE,...,IMAGERY/OTHER,2015-06-19T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,312,PK,08,2201290,21,NOT AVAILABLE
2,-69.971880,43.908147,2004,230378023129,HARRIET BEECHER STOWE ELEMENTARY,44 MCKEEN STREET,BRUNSWICK,ME,4011,NOT AVAILABLE,...,IMAGERY,2014-05-07T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,739,02,05,2303780,52,NOT AVAILABLE
3,-89.542799,32.728496,2005,280252001118,LEAKE CENTRAL ELEMENTARY SCHOOL,603 HWY. 16 WEST,CARTHAGE,MS,39051,NOT AVAILABLE,...,IMAGERY/OTHER,2010-07-06T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,1159,PK,05,2802520,63,NOT AVAILABLE
4,-94.361775,39.364359,2006,291645000891,KEARNEY ELEM.,902 S JEFFERSON,KEARNEY,MO,64060,8518,...,IMAGERY/OTHER,2016-07-18T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,274,KG,05,2916450,22,NOT AVAILABLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102365,-83.085229,42.320632,102216,260032201947,COVENANT HOUSE ACADEMY DETROIT - SOUTHWEST SITE,1450 25TH ST,DETROIT,MI,48216,1404,...,IMAGERY,2016-01-13T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,3,202,09,12,2600322,7,NOT AVAILABLE
102366,-83.272599,42.062038,102217,260198003940,FRED W. RITTER ELEMENTARY SCHOOL,5650 CARLETON ROCKWOOD RD,SOUTH ROCKWOOD,MI,48179,9764,...,IMAGERY,2010-06-04T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,291,KG,04,2601980,17,NOT AVAILABLE
102367,-88.914089,30.436478,102218,280177000284,DIBERVILLE ELEM,4540 BRODIE ROAD,DIBERVILLE,MS,39540,NOT AVAILABLE,...,IMAGERY,2013-05-28T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,761,KG,03,2801770,40,NOT AVAILABLE
102368,-94.558365,39.187941,102219,292280001267,DAVIDSON ELEM.,5100 N HIGHLAND,KANSAS CITY,MO,64118,6049,...,IMAGERY/OTHER,2016-07-18T00:00:00.000Z,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,1,404,PK,05,2922800,22,NOT AVAILABLE


In [ ]:
grades_dict = {'N': -2,'PK': -1,'KG': 0,'01': 1,'02': 2,'03': 3,'04': 4,'05': 5,'06': 6,'07': 7,'08': 8,'09': 9,'10': 10,'11': 11,'12': 12,'13': 13,'AE': 14,'UG': 15}

In [ ]:
temp = df.copy()
temp = temp.drop(["X", "Y", "NCESID", "ADDRESS", "NAME", "SHELTER_ID", "OBJECTID", "VAL_METHOD", "TELEPHONE", "WEBSITE", "SOURCEDATE", "VAL_DATE",
                  "NAICS_CODE", "NAICS_DESC", "SOURCE"], axis=1)
temp = temp.rename({
    col:col.lower()
    for col in temp
}, axis=1)

# Cleaning
temp['enrollment'] = temp['enrollment'].apply(lambda x: x if x > 0 else None)
temp['population'] = temp['population'].apply(lambda x: x if x > 0 else None)
temp['ft_teacher'] = temp['ft_teacher'].apply(lambda x: x if x > 0 else None)
temp.loc[temp['st_grade'] == '13', 'end_grade'] = 'AE'
temp.loc[temp['st_grade'] == '13', 'st_grade'] = 'AE'
temp['st_grade'] = temp['st_grade'].map(grades_dict) # convert to numbers
temp['end_grade'] = temp['end_grade'].map(grades_dict) # convert to numbers
temp = temp[temp['country'] == 'USA'].drop(["country"], axis=1)


# New Columns
temp['enrollment_rate'] = temp['enrollment'] / temp['population']
temp['student_teacher_ratio'] = temp['population'] / temp['ft_teacher']
temp['span'] = temp['end_grade'] - temp['st_grade'] + 1

# Final Touches
temp = temp[['state', 'county', 'city', 'countyfips', 'districtid', 'population', 'enrollment', 'enrollment_rate',
             'st_grade', 'end_grade', "span", 'ft_teacher', "student_teacher_ratio", 'type', 'status',
             'level_', 'zip', 'zip4', 'latitude', 'longitude',]]

temp = temp.reset_index(drop=True)
temp

,state,county,city,countyfips,districtid,population,enrollment,enrollment_rate,st_grade,end_grade,span,ft_teacher,student_teacher_ratio,type,status,level_,zip,zip4,latitude,longitude
0,FL,VOLUSIA,NEW SMYRNA,12127,1201920,235.0,222.0,0.944681,0,5,6,13.0,18.076923,1,1,1,32168,8762,29.022271,-81.050895
1,LA,RAPIDES,WOODWORTH,22079,2201290,333.0,312.0,0.936937,-1,8,10,21.0,15.857143,1,1,1,71485,NOT AVAILABLE,31.180659,-92.507288
2,ME,CUMBERLAND,BRUNSWICK,23005,2303780,791.0,739.0,0.934260,2,5,4,52.0,15.211538,1,1,1,4011,NOT AVAILABLE,43.908147,-69.971880
3,MS,LEAKE,CARTHAGE,28079,2802520,1222.0,1159.0,0.948445,-1,5,7,63.0,19.396825,1,1,1,39051,NOT AVAILABLE,32.728496,-89.542799
4,MO,CLAY,KEARNEY,29047,2916450,296.0,274.0,0.925676,0,5,6,22.0,13.454545,1,1,1,64060,8518,39.364359,-94.361775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100876,MI,WAYNE,DETROIT,26163,2600322,209.0,202.0,0.966507,9,12,4,7.0,29.857143,4,1,3,48216,1404,42.320632,-83.085229
100877,MI,MONROE,SOUTH ROCKWOOD,26115,2601980,308.0,291.0,0.944805,0,4,5,17.0,18.117647,1,1,1,48179,9764,42.062038,-83.272599
100878,MS,HARRISON,DIBERVILLE,28047,2801770,801.0,761.0,0.950062,0,3,4,40.0,20.025000,1,1,1,39540,NOT AVAILABLE,30.436478,-88.914089
100879,MO,CLAY,KANSAS CITY,29047,2922800,426.0,404.0,0.948357,-1,5,7,22.0,19.363636,1,1,1,64118,6049,39.187941,-94.558365


In [ ]:
temp.to_csv(drive_path + 'usa_schools.csv', index=True, index_label='id')